In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
import pandas as pd

In [2]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
import numpy as np

def knn(nutrients, dataset):
    # Load the dataset into a pandas DataFrame
    ds = pd.read_csv(dataset)
    ds = ds.dropna(subset=['name'])

    # Extract features (calories, carb, fat, protein, sugar) and target variable (name)
    x = ds[['calories', 'carb', 'fat', 'protein', 'sugar']]
    y = ds['name']

    # Impute missing values in features
    imputer = SimpleImputer(strategy='mean')
    x_imputed = imputer.fit_transform(x)

    # Define a weight function that assigns weights based on distance and nutrient importance
    def weight_function(distances):
        # Assign weights based on the specified nutrient weights
        weights = {
        }
        weights['calories'] = 1
        weights['carb'] = 4
        weights['fat'] = 9
        weights['protein'] = 4
        weights['sugar'] = 1

        # Calculate the weighted distances
        for i in range(len(distances)):
            distance = distances[i]
            nutrient_weights = []
            for nutrient in nutrients:
                nutrient_weights.append(weights[nutrient])

            # Apply the weight factor to the distance
            weighted_distance = distance * np.prod(nutrient_weights)
            distances[i] = weighted_distance

        return distances

    # Create an instance of the KNeighborsClassifier with weights
    model = KNeighborsClassifier(n_neighbors=2, weights=weight_function)

    # Fit the model on the imputed dataset
    model.fit(x_imputed, y)

    # Create a new DataFrame with the given nutrients
    nutrients_df = pd.DataFrame([nutrients])

    # Ensure the input for imputation has the same number of features as the training data
    nutrients_df_imputed = imputer.transform(nutrients_df)

    # Predict the meal names based on the imputed nutrients
    predicted_meal = model.predict(nutrients_df_imputed)

    # Get the nutrients of the predicted meals
    predicted_nutrients = ds.loc[ds['name'].isin(predicted_meal), ['calories', 'carb', 'fat', 'protein', 'sugar']].values

    return predicted_meal[0], predicted_nutrients[0]

In [3]:
def knn(nutrients, dataset):
    # Load the dataset into a pandas DataFrame
    ds = pd.read_csv(dataset)
    ds = ds.dropna(subset=['name'])

    # Extract the features (calories, carb, fat, protein, sugar) and the target variable (name)
    x = ds[['calories', 'carb', 'fat', 'protein', 'sugar']]
    y = ds['name']

    # Impute missing values in features
    imputer = SimpleImputer(strategy='mean')
    x_imputed = imputer.fit_transform(x)

    # Create an instance of the KNeighborsClassifier
    model = KNeighborsClassifier(n_neighbors=2)

    # Fit the model on the imputed dataset
    model.fit(x_imputed, y)

    # Create a new DataFrame with the given nutrients
    nutrients_df = pd.DataFrame([nutrients])

    # Ensure the input for imputation has the same number of features as the training data
    nutrients_df_imputed = imputer.transform(nutrients_df)

    # Predict the meal names based on the imputed nutrients
    predicted_meal = model.predict(nutrients_df_imputed)

    # Get the nutrients of the predicted meals
    predicted_nutrients = ds.loc[ds['name'].isin(predicted_meal), ['calories', 'carb', 'fat', 'protein', 'sugar']].values

    return predicted_meal[0], predicted_nutrients[0]

In [4]:
def load_dishes_csv(dish_type):
    # Load dishes of a specific type from its corresponding CSV
    file_path = f"Datasets/{dish_type}.csv"
    return file_path

In [5]:
dish_types = ['Drinks', 'Sandwiches', 'Desserts', 'Main dishes', 'Salads', 'Pasta', 'Appetizers', 'Soups', 'Mostly meat']

In [6]:
nutrients = {
    "calories": 1775,
    "carb": 150,
    "fat": 75,
    "protein": 125,
    "sugar": 15
}

In [7]:
meal_ratios = {"breakfast": 0.3, "lunch": 0.4, "snacks": 0.1, "dinner": 0.2}

In [8]:
distribution_ratios = {
    "breakfast": {"Drinks": 0.2, "Sandwiches": 0.4, "Desserts": 0.4},
    "lunch": {"Main dishes": 0.5, "Salads": 0.1, "Pasta": 0.4},
    "snacks": {"Appetizers": 0.5, "Desserts": 0.4, "Drinks": 0.1},
    "dinner": {"Main dishes": 0.5, "Soups": 0.3, "Drinks": 0.2}
}

In [9]:
predicted_dishes = {}
sum = 0
for meal, meal_ratio in meal_ratios.items():
    print(f"Meal: {meal}")
    
    # Get distribution ratios for the current meal
    meal_distribution_ratios = distribution_ratios.get(meal, {})
    
    # Calculate nutrients for each dish in the meal
    for dish, dish_ratio in meal_distribution_ratios.items():
        # Calculate nutrients for the current dish based on ratios
        dish_nutrients = {key: value * meal_ratio * dish_ratio for key, value in nutrients.items()}
        dish_path = load_dishes_csv(dish)
        
        predicted_meal, predicted_nutrients = knn(dish_nutrients, dish_path)
        
        # Store the predicted meal and nutrients for the current dish
        predicted_dishes[dish] = {
            'predicted_meal': predicted_meal,
            'predicted_nutrients': predicted_nutrients
        }
        
        # Print the results for each dish
        print(f"{dish} - Predicted Meal: {predicted_meal}, Predicted Nutrients: {predicted_nutrients}")
        sum += predicted_nutrients
        
    print(sum)      
    print("\n")
   

Meal: breakfast
Drinks - Predicted Meal: Chia Seed Pudding, Predicted Nutrients: [105.    9.8   5.9   3.    0.2]
Sandwiches - Predicted Meal: Corned Beef Sandwich, Predicted Nutrients: [215.   15.9  11.   13.1   1.8]
Desserts - Predicted Meal: Breakfast Cookies, Predicted Nutrients: [215.   28.2   9.7   7.    6.4]
[535.   53.9  26.6  23.1   8.4]


Meal: lunch
Main dishes - Predicted Meal: Cheese On Toast, Predicted Nutrients: [363.   25.8  21.3  17.4   3.8]
Salads - Predicted Meal: Buffalo Chicken Salad, Predicted Nutrients: [71.   4.2  4.3  4.5  1.4]
Pasta - Predicted Meal: Creamy Cheese And Lemon Ravioli, Predicted Nutrients: [290.   32.2  13.2  12.9   1.5]
[1259.   116.1   65.4   57.9   15.1]


Meal: snacks
Appetizers - Predicted Meal: Cauliflower Tots, Predicted Nutrients: [172.   14.7   8.5   9.4   2.1]
Desserts - Predicted Meal: Overnight Chia Seed Pudding, Predicted Nutrients: [63.   8.   3.1  1.7  1.6]
Drinks - Predicted Meal: Tropical Fruit Smoothie, Predicted Nutrients: [17. 